In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np

#hyper parameters
input_size = 1024
batch_size = 100
learning_rate = 0.001
momentum = 0.9
num_class = 10
num_epoch = 10

# cifar10 dataset
train_dataset = dsets.CIFAR10(root='./data', 
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = dsets.CIFAR10(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)


In [ ]:
class CIFAR10(nn.Module):
    def __init__(self):
        super(CIFAR10, self).__init__()
        # input channel, output channel, kernel_size, 
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_class)
     
    def forward(self, x):
        # convolution step
        l1 = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        l2 = F.max_pool2d(F.relu(self.conv2(l1)), 2)
        
        # flatten & fully connected layer
        l2 = l2.view(-1, 16 * 5 * 5)
        fully_contd1 = F.relu(self.fc1(l2))
        fully_contd2 = F.relu(self.fc2(fully_contd1))
        out = self.fc3(fully_contd2)
        return out
    
    '''
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    '''
        

In [ ]:
CF10 = CIFAR10()
CF10.cuda(0)
print (CF10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CF10.parameters(), lr=learning_rate)

In [ ]:
# training step

for epoch in range(num_epoch):
    for i, (image, label) in enumerate(train_loader):
        images = Variable(image.cuda()) #.view(-1, 3, 32, 32)
        labels = Variable(label.cuda())
        
        optimizer.zero_grad()
        output = CF10(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        if(i+1) % 100 == 0:
            print('epoch: [%d/%d], step: [%d/%d], Loss: %.4f' % (epoch+1, num_epoch, i+1, len(train_dataset)//batch_size, loss.data[0]))

epoch: [1/10], step: [100/500], Loss: 1.9530
epoch: [1/10], step: [200/500], Loss: 1.9035
epoch: [1/10], step: [300/500], Loss: 1.6055
epoch: [1/10], step: [400/500], Loss: 1.6775
epoch: [1/10], step: [500/500], Loss: 1.5833
epoch: [2/10], step: [100/500], Loss: 1.4339
epoch: [2/10], step: [200/500], Loss: 1.6821
epoch: [2/10], step: [300/500], Loss: 1.5873
epoch: [2/10], step: [400/500], Loss: 1.4489
epoch: [2/10], step: [500/500], Loss: 1.4384
epoch: [3/10], step: [100/500], Loss: 1.4674
epoch: [3/10], step: [200/500], Loss: 1.4580
epoch: [3/10], step: [300/500], Loss: 1.3291
epoch: [3/10], step: [400/500], Loss: 1.3230
epoch: [3/10], step: [500/500], Loss: 1.2284
epoch: [4/10], step: [100/500], Loss: 1.3777
epoch: [4/10], step: [200/500], Loss: 1.2729
epoch: [4/10], step: [300/500], Loss: 1.1461
epoch: [4/10], step: [400/500], Loss: 1.3385
epoch: [4/10], step: [500/500], Loss: 1.5855
epoch: [5/10], step: [100/500], Loss: 1.1909
epoch: [5/10], step: [200/500], Loss: 1.2089
epoch: [5/

In [52]:
total = 0
correct = 0

for image, label in test_loader:
    images = Variable(image.cuda())
    #label = Variable(label.cuda())
    _, predicted = torch.max(CF10(images).data, 1)
    total += label.size(0)
    correct += (predicted.cpu() == label).sum()
print('accuracy of the model : %d %%' % (100*correct/total))

accuracy of the model : 29 %
